In [39]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ecommerce-data/data.csv


In [40]:
dataset_path = "/kaggle/input/ecommerce-data"
print("Files in the dataset:")
print(os.listdir(dataset_path)) # os.listdir() will show you all available files

Files in the dataset:
['data.csv']


In [41]:
# Only Loads the data -- encoding = 'latin1' because file supports special chars
file_name = "data.csv"
df = pd.read_csv(os.path.join(dataset_path, file_name), encoding = 'latin1')

In [42]:
print("\nFirst 5 rows:")
print(df.head())


First 5 rows:
  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

      InvoiceDate  UnitPrice  CustomerID         Country  
0  12/1/2010 8:26       2.55     17850.0  United Kingdom  
1  12/1/2010 8:26       3.39     17850.0  United Kingdom  
2  12/1/2010 8:26       2.75     17850.0  United Kingdom  
3  12/1/2010 8:26       3.39     17850.0  United Kingdom  
4  12/1/2010 8:26       3.39     17850.0  United Kingdom  


In [43]:
# Data Struture
print("\n Dataset Information:")
print(df.info())


 Dataset Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB
None


In [44]:
# Basic Statistics
print("\nBasic Statistics:")
print(df.describe(include='all'))


Basic Statistics:
       InvoiceNo StockCode                         Description       Quantity  \
count     541909    541909                              540455  541909.000000   
unique     25900      4070                                4223            NaN   
top       573585    85123A  WHITE HANGING HEART T-LIGHT HOLDER            NaN   
freq        1114      2313                                2369            NaN   
mean         NaN       NaN                                 NaN       9.552250   
std          NaN       NaN                                 NaN     218.081158   
min          NaN       NaN                                 NaN  -80995.000000   
25%          NaN       NaN                                 NaN       1.000000   
50%          NaN       NaN                                 NaN       3.000000   
75%          NaN       NaN                                 NaN      10.000000   
max          NaN       NaN                                 NaN   80995.000000   

        

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


In [45]:
# we choose id because id is main point in any dataset 
missing_id = df['CustomerID'].isnull().sum()
print(f"Missing CustomerID values: {missing_id}")

df = df.dropna(subset=['CustomerID'])

Missing CustomerID values: 135080


In [46]:
# now remove duplicates from InvoiceNo + StockCode 
# because now invoice and stockcode are second ids 

# print no.of rows before removing duplicates
print(f"Number of rows before removing duplicates: {df.shape[0]}")

# Drop duplicates = remove
df = df.drop_duplicates(subset =['InvoiceNo', 'StockCode'], keep = 'first')
print(f"Rows after removing Duplicates: {df.shape[0]}")


Number of rows before removing duplicates: 406829
Rows after removing Duplicates: 396681


In [47]:
# Outliers

print("Quantity stats:")
print(df['Quantity'].describe())

print("\nUnitPrice stats:")
print(df['UnitPrice'].describe())

# thresholds for outliers 
max_quantity = 1000
max_unit_price = 1000

min_quantity = 0
min_unit_price = 0

df = df[(df['Quantity'] <= max_quantity) & (df['UnitPrice'] <= max_unit_price)]
df = df[(df['Quantity'] >= min_quantity) & (df['UnitPrice'] >= min_unit_price)]
print(f"Rows after removing outliers: {df.shape[0]}")

Quantity stats:
count    396681.000000
mean         12.278299
std         251.820419
min      -80995.000000
25%           2.000000
50%           5.000000
75%          12.000000
max       80995.000000
Name: Quantity, dtype: float64

UnitPrice stats:
count    396681.000000
mean          3.454548
std          69.752711
min           0.000000
25%           1.250000
50%           1.950000
75%           3.750000
max       38970.000000
Name: UnitPrice, dtype: float64
Rows after removing outliers: 387752


In [48]:
# convert InvoiceDate into datetime format 
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], errors='coerce')
print(df['InvoiceDate'].head())

0   2010-12-01 08:26:00
1   2010-12-01 08:26:00
2   2010-12-01 08:26:00
3   2010-12-01 08:26:00
4   2010-12-01 08:26:00
Name: InvoiceDate, dtype: datetime64[ns]


In [49]:
# we did this after converting invoicedate so it won't drop all dates 
df = df.dropna(subset=['InvoiceDate'])

In [52]:
# reset index after cleaning
df_clean = df.reset_index(drop=True, inplace=True)
print(df.index)

RangeIndex(start=0, stop=387752, step=1)


In [53]:
df.sample(50)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
214825,564083,22411,JUMBO SHOPPER VINTAGE RED PAISLEY,10,2011-08-23 08:54:00,2.08,17469.0,United Kingdom
208623,563444,23132,SMALL IVORY HEART WALL ORGANISER,1,2011-08-16 13:50:00,5.75,16033.0,United Kingdom
82533,547357,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-03-22 12:15:00,4.95,17613.0,United Kingdom
210511,563614,22557,PLASTERS IN TIN VINTAGE PAISLEY,96,2011-08-18 08:51:00,1.45,12415.0,Australia
121009,552329,23207,LUNCH BAG ALPHABET DESIGN,10,2011-05-09 10:06:00,1.65,13492.0,Switzerland
244029,567462,21213,PACK OF 72 SKULL CAKE CASES,1,2011-09-20 12:35:00,0.55,14446.0,United Kingdom
347641,577497,23289,DOLLY GIRL CHILDRENS BOWL,4,2011-11-20 12:22:00,1.25,17555.0,United Kingdom
220792,564749,POST,POSTAGE,2,2011-08-30 08:15:00,18.00,12708.0,Germany
116922,551869,22998,TRAVEL CARD WALLET KEEP CALM,48,2011-05-04 16:36:00,0.42,16767.0,United Kingdom
90317,548308,22982,PANTRY PASTRY BRUSH,6,2011-03-30 12:00:00,1.25,17220.0,United Kingdom


In [54]:
df.to_csv("cleaned_data.csv", index=False)